In [1]:
import pickle
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from sklearn.metrics import (accuracy_score, auc, average_precision_score,
                             classification_report, confusion_matrix,
                             precision_recall_curve, precision_score,
                             recall_score, roc_curve)
import kds
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array


In [2]:
test = pd.read_pickle('ArchivosUtiles/testingAnimal.pkl')

In [3]:
X_test = test['Imagen']
y_test = test['Animal']

In [4]:
X_test_tf = tf.convert_to_tensor(np.array([img_to_array(img) for img in X_test]))

In [5]:
model = load_model('ModelosFinales/modeloAnimalVGG16.h5')

In [6]:
y_proba = model.predict(X_test_tf)
y_pred = (y_proba >= 0.5).astype(int)

2023-12-15 19:55:14.465967: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


48/48 [==============================] - 48s 971ms/step


In [7]:
def optimizar_umbral(y_test, y_proba, tarea, CFP=1000, CFN=300, CV=24.42, TV=0.579, img_dir='img_temp'):
    # Crear directorio para imágenes temporales
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)

    #  Inicializacion de variables
    limInf = 0
    limSup = 1
    CostoMin = float('inf')
    Vop = 0
    # Determinacion de costos.
    if tarea == 'especie':
        CE = CFP + CFN
    else:
        CE = (CFP + CFN) / 2

    contador = 0
    for umbral_inf in np.arange(0.01, 0.5, 0.01):
        for umbral_sup in np.arange(0.5, 0.99, 0.01):
            # Determina la cantidad de imagenes que caen entre los umbrales.
            V = np.sum((y_proba >= umbral_inf) & (y_proba <= umbral_sup))
            #  Costo generado por la validacion.
            C = V * CV 
            #  Costo de Error de Validacion
            C += V * TV * CE
            #  Determina el error que conlleva no validar fuera de los umbrales.
            if tarea == 'presencia':
                #  Errores por Falsos Positivos fuera de los umbrales.
                FP = np.sum((y_proba < umbral_inf) | (y_proba > umbral_sup) & (y_test == 0))
                #  Errores por Falsos Negativos fuera de los umbrales.
                FN = np.sum((y_proba < umbral_inf) | (y_proba > umbral_sup) & (y_test == 1))
                #  Costos generados por errores de clasificacion.
                C += FP * CFP + FN * CFN
            else:
                # Errores generados.
                E = np.sum((y_proba < umbral_inf) | (y_proba > umbral_sup))
                # Costo generado por errores de clasificacion.
                C += E * CE
            # En el caso de que el costo sea menor al minimo actual, se actualiza el minimo.
            if C < CostoMin:
                Vop = V
                CostoMin = C
                limInf = umbral_inf
                limSup = umbral_sup

            y_pred = (y_proba >= 0.5).astype(int)
            # Crear colores basados en si son errores y su posición relativa a los umbrales
            colores = ['#96482C' if ((prob < umbral_inf or prob > umbral_sup) and real != clase)
                       else '#4E548D' if ((prob >= umbral_inf and prob <= umbral_sup))
                       else '#87A26B' for prob, real, clase in zip(y_proba, y_test, y_pred)]

            # Guardar imagen
            plt.figure(figsize=(10, 6))
            plt.scatter(y_proba, range(len(y_proba)), color=colores, alpha=0.7)
            plt.axvspan(umbral_inf, umbral_sup, alpha=0.2, color='#272E72')
            plt.title(f'Distribución de Probabilidades - Umbral: {round(umbral_inf, 2)} - {round(umbral_sup, 2)} - Costo: {round(C, 2)}')
            plt.xlabel('Probabilidades')
            plt.ylabel('Índices')
            plt.tight_layout()
            plt.savefig(f"{img_dir}/img_{contador}.png")
            plt.close()
            contador += 1


In [8]:
optimizar_umbral(y_test, y_proba.ravel(),'presencia')

In [9]:
import imageio

contador = 2400
imagenes = [imageio.imread(f"img_temp/img_{i}.png") for i in range(contador)]
imageio.mimsave('animacion.gif', imagenes, fps=10)  # Ajusta el fps según lo desees

/var/folders/6c/17qbm4nd5318hd4bcrx68wjh0000gn/T/ipykernel_62452/2176571065.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imagenes = [imageio.imread(f"img_temp/img_{i}.png") for i in range(contador)]
